# 🛡️ SafeChatBot - Chatbot with Moderation Filters
This Google Colab notebook sets up a chatbot interface using OpenAI's GPT and moderation filters for safe input/output.

In [ ]:
# 📦 Install dependencies
!pip install openai gradio

In [ ]:
# 🔐 Set your OpenAI API key
import openai
openai.api_key = "your-api-key"  # Replace this with your actual key

In [ ]:
# 🧹 Define moderation functions
import re

def is_input_safe(user_input):
    banned_keywords = ["hate", "kill", "racist", "bomb", "terrorist"]
    pattern = re.compile("|".join(re.escape(word) for word in banned_keywords), re.IGNORECASE)
    return not bool(pattern.search(user_input))

def censor_response(text):
    banned_keywords = ["hate", "kill", "racist"]
    for word in banned_keywords:
        text = re.sub(rf"(?i)\\b{word}\\b", "[CENSORED]", text)
    return text

In [ ]:
# 🤖 Define chatbot logic
import gradio as gr

def generate_reply(user_input, history=[]):
    if not is_input_safe(user_input):
        return "⚠️ Input blocked due to unsafe content", history
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}]
    )
    reply = response['choices'][0]['message']['content']
    reply = censor_response(reply)
    
    history.append((user_input, reply))
    return "", history

In [ ]:
# 🚀 Launch Gradio chatbot
chatbot = gr.ChatInterface(fn=generate_reply, title="🛡️ SafeChatBot")
chatbot.launch()